In [1]:
import requests
import numpy as np
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from tqdm import tqdm_notebook
import socks
import socket
import math
import re
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataFile = open('id_films_2.txt')
id_films = []
for line in dataFile:
    id_films.append(line.replace('\n',''))

In [3]:
id_films[:10]

['/film/586498/',
 '/film/899325/',
 '/film/596025/',
 '/film/839374/',
 '/film/1338068/',
 '/film/1152875/',
 '/film/1047033/',
 '/film/1112633/',
 '/film/1257283/',
 '/film/2500772/']

парсинг запускался несколько раз для разных списков с фильмами:
1. первоначально был взят топ250 -  с одной страницы сайта руками были взяты id фильмов для дальнейшего парсинга рецензий.

но! этого показалось мало + разброс отзывов был такой что были в основном положительные.

2. далее взялся список из 744 самых худших фильмов (рейтинг 0 - 4.1)

но! этого показалось мало... id брались также как в первый раз.

3. взялся список из 916 фильмов со "средним" рейтингом (4.2 - 6)

но! этого показалось мало... id брались также как в первые два раза.

4. на сайте таки был найден навигатор, используя который удалось спарсить намного больше id фильмов - 8973 фильма - описан в файле cource_project_parser_ids. в данном случае парсер запускался на каждые 1000 id. далее в файле сохранен запуск для последних 973 фильма.


In [4]:
id_film = []
for x in id_films:
    id_film.append(int(x.split('/')[2]))

In [5]:
len(id_film)

8973

In [6]:
id_film = id_film[-973:]

In [7]:
len(id_film)

973

In [7]:
def checkBot(soup):
    if soup.find(lambda tag: tag.name == 'input' and tag.get('class') == ['CheckboxCaptcha-Button']):
        return 1
    else:
        return 0

In [8]:
def checkError(soup):
    if soup.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ['error-page']):
        return 1
    else:
        return 0

In [9]:
def getPageLinks(id_film, browser):
    
    sleep_time = 10.0
    
    time.sleep(sleep_time)
    
    data_row = []
    
    page_link = 'https://www.kinopoisk.ru/film/{}/reviews/ord/date/status/all/perpage/200/'.format(id_film)
#     response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
#     html = response.content
#     soup = BeautifulSoup(html,'html.parser')
#     browser=webdriver.Firefox()
    browser.get(page_link)
    soup=BeautifulSoup(browser.page_source,'html.parser')
    if checkBot(soup):
        #print('bot')
        return data_row
    
    if checkError(soup):
        #print('error pages')
        return data_row
    
    pages_reviews = []

    count_reviews = int(((soup.find(lambda tag: tag.name == 'li' and tag.get('class') == ['all'])).find(lambda tag: tag.name == 'b')).text)
    #print(count_reviews)
    if count_reviews == 0:
        return data_row
    elif count_reviews <= 200 and count_reviews > 0:
        data_row = getRewiewData(soup)
    else:
        data_row = getRewiewData(soup)
        
        count_pages = math.ceil(count_reviews / 200)
        #print(count_pages)
        for i in range(2, count_pages + 1):
            #data_row_temp = []
            time.sleep(sleep_time)
            page_link = 'https://www.kinopoisk.ru/film/{}/reviews/ord/date/status/all/perpage/200/page/{}/'.format(id_film, i)
#             response = requests.get(page_link, headers={'User-Agent': UserAgent().Firefox})
#             html = response.content
#             soup = BeautifulSoup(html,'html.parser')
            browser.get(page_link)
            soup=BeautifulSoup(browser.page_source,'html.parser')
            if checkBot(soup):
                #print('bot')
                return data_row
            if checkError(soup):
                #print('error pages')
                return data_row
            #data_row_temp = getRewiewData(soup)
            data_row.extend(getRewiewData(soup))
    return data_row

почему было принято решение уйти от библиотеки requests - похоже после выполнения домашней работы 5 (по сути данный проект это продолжение данной работы) кинопоиск что-то понял, в этот раз с тем же самым парсером я все время попадал на капчу. поэтому было принято решение использовать selenium и эмулировать работы браузера. изменения в коде были минимальны. а спарсить данных удалось намного больше.

In [10]:
def getRewiewData(soup):
    
    type_comment = []
    date_comment = []
    text_comment = []
    data_row = []
      
    type_comment = [d[1] for d in [x.attrs['class'] for x in soup.findAll(lambda tag: tag.name == 'div' and tag.get('itemprop') == 'reviews')]]
    date_comment = [x.text for x in soup.findAll(lambda tag: tag.name == 'span' and tag.get('class') == ['date'])]
    text_comment = [x.text for x in soup.findAll(lambda tag: tag.name == 'span' and tag.get('class') == ['_reachbanner_'])]
    name_movie = [x.text for x in soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['breadcrumbs__link'])]
    

    for i, j, k in zip(type_comment, date_comment, text_comment):
        data_row.append([i, j, k, name_movie[0]])
    
    return data_row

In [11]:
data_df = pd.DataFrame(columns=['type_comment', 'date_comment', 'text_comment', 'name_movie'])

In [12]:
exeptions = []
browser=webdriver.Firefox()

for page_number in tqdm_notebook(id_film):
    try:
        data_row = getPageLinks(page_number, browser)
        df = pd.DataFrame(data_row,columns=['type_comment', 'date_comment', 'text_comment', 'name_movie'])
        data_df = data_df.append(df)
    except:
        exeptions.append(page_number)
        continue

  0%|          | 0/973 [00:00<?, ?it/s]

In [13]:
data_df.head()

,type_comment,date_comment,text_comment,name_movie
0,bad,14 июня 2021 | 20:42,Долгое время являлся большим фанатом фильмов К...,"Гуляй, Вася! Свидание на Бали"
1,bad,07 мая 2021 | 23:56,Ну сколько раз мир повторял киноделам: если по...,"Гуляй, Вася! Свидание на Бали"
2,good,20 апреля 2021 | 11:04,Глубина без грамма серьёзности или обман без г...,"Гуляй, Вася! Свидание на Бали"
3,good,20 апреля 2021 | 09:21,Если первую часть можно назвать комедийной мел...,"Гуляй, Вася! Свидание на Бали"
4,bad,18 апреля 2021 | 12:48,"Среди киноманов уже давно бытует мнение, что п...","Гуляй, Вася! Свидание на Бали"


In [14]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28345 entries, 0 to 111
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type_comment  28345 non-null  object
 1   date_comment  28345 non-null  object
 2   text_comment  28345 non-null  object
 3   name_movie    28345 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [16]:
with open('comments_add_9.json', 'w', encoding='utf-8') as file:
    data_df.to_json(file, force_ascii=False, orient="records")